In [1]:
from __future__ import print_function
import tensorflow as tf

/Users/OG/Desktop/Study/jupyter/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/OG/Desktop/Study/jupyter/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../data/", one_hot=True)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../data/t10k-images-idx3-ubyte.gz
Extracting ../../data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [46]:
# Define some parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

# Where to save TensorBoard model summaries
LOG_DIR = "logs/RNN_with_summaries"

In [47]:
# Create placeholders for inputs, labels
_inputs = tf.placeholder(tf.float32,
                         shape=[None, time_steps, element_size],
                         name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')


In [48]:
# This helper function taken from official TensorFlow documentation,
# simply add some ops that take care of logging summaries
# for tensorboard graph
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [49]:
# Weights and bias for input and hidden layer
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

In [50]:
def rnn_step(previous_hidden_state, x):

        current_hidden_state = tf.tanh(
            tf.matmul(previous_hidden_state, Wh) +
            tf.matmul(x, Wx) + b_rnn)
        # tanh((prev_hidden_state*w)+(x*w)+b)
        return current_hidden_state

In [51]:
# Processing inputs to work with scan function
# Current input shape: (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])
# Current input shape now: (time_steps,batch_size, element_size)
print("processed_input: "+str(processed_input))


initial_hidden = tf.zeros([batch_size, hidden_layer_size])
# Getting all state vectors across time
# doesn't need loop 
# get every result from a loop 
all_hidden_states = tf.scan(rnn_step,
                            processed_input,
                            initializer=initial_hidden,
                            name='states')
print("all_hidden_states: "+str(all_hidden_states))


processed_input: Tensor("transpose_4:0", shape=(28, ?, 28), dtype=float32)
all_hidden_states: Tensor("states_4/TensorArrayStack/TensorArrayGatherV3:0", shape=(28, 128, 128), dtype=float32)


In [52]:
# Weights and bias for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                             mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                             mean=0, stddev=.01))
        variable_summaries(bl)

In [53]:
# Apply linear layer to state vector
def get_linear_layer(hidden_state):
    print("hidden_state: "+str(hidden_state))
    return tf.matmul(hidden_state, Wl) + bl

with tf.name_scope('linear_layer_weights') as scope:
    # Iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    print("all_outputs: "+str(all_outputs))
    # Get Last output -- h_28 because it's input layer
    output = all_outputs[-1]
    print("output: "+str(output))
    tf.summary.histogram('outputs', output)

hidden_state: Tensor("linear_layer_weights_15/map/while/TensorArrayReadV3:0", shape=(128, 128), dtype=float32)
all_outputs: Tensor("linear_layer_weights_15/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(28, 128, 10), dtype=float32)
output: Tensor("linear_layer_weights_15/strided_slice:0", shape=(128, 10), dtype=float32)


In [ ]:
# question: why is all_outputs (128,10) and not (128,128)?
# is layer 28 > 128 > 10 ??

In [54]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    # Using RMSPropOptimizer
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))

    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries
merged = tf.summary.merge_all()

In [55]:
# Get a small test set
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

In [56]:
test_data.shape

(128, 28, 28)

In [42]:
with tf.Session() as sess:
    # Write summaries to LOG_DIR -- used by TensorBoard
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train',
                                         graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test',
                                        graph=tf.get_default_graph())

    sess.run(tf.global_variables_initializer())

    for i in range(10000):

            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Reshape data to get 28 sequences of 28 pixels
            batch_x = batch_x.reshape((batch_size, time_steps, element_size))
            summary, _ = sess.run([merged, train_step],
                                  feed_dict={_inputs: batch_x, y: batch_y})
            # Add to summaries
            train_writer.add_summary(summary, i)

            if i % 1000 == 0:
                acc, loss, = sess.run([accuracy, cross_entropy],
                                      feed_dict={_inputs: batch_x,
                                                 y: batch_y})
                print("Iter " + str(i) + ", Minibatch Loss= " +
                      "{:.6f}".format(loss) + ", Training Accuracy= " +
                      "{:.5f}".format(acc))
            if i % 100 == 0:
                # Calculate accuracy for 128 mnist test images and
                # add to summaries
                summary, acc = sess.run([merged, accuracy],
                                        feed_dict={_inputs: test_data,
                                                   y: test_label})
                test_writer.add_summary(summary, i)

    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                             y: test_label})
    print("Test Accuracy:", test_acc)


Iter 0, Minibatch Loss= 2.302179, Training Accuracy= 12.50000
Iter 1000, Minibatch Loss= 1.448546, Training Accuracy= 49.21875
Iter 2000, Minibatch Loss= 0.684637, Training Accuracy= 81.25000
Iter 3000, Minibatch Loss= 0.296832, Training Accuracy= 91.40625
Iter 4000, Minibatch Loss= 0.067264, Training Accuracy= 98.43750
Iter 5000, Minibatch Loss= 0.089733, Training Accuracy= 98.43750
Iter 6000, Minibatch Loss= 0.026686, Training Accuracy= 100.00000
Iter 7000, Minibatch Loss= 0.028104, Training Accuracy= 100.00000
Iter 8000, Minibatch Loss= 0.022553, Training Accuracy= 100.00000
Iter 9000, Minibatch Loss= 0.033719, Training Accuracy= 99.21875
Test Accuracy: 97.65625


## 텐서플로 내장 RNN

In [92]:
tf.reset_default_graph()
# LOG_DIR = "logs/RNN_with_summaries_2" # there is no summaries in here

In [93]:
_inputs = tf.placeholder(tf.float32,
                         shape=[None, time_steps, element_size],
                         name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='inputs')

In [94]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                     mean=0, stddev=.01))
bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))

In [97]:
last_rnn_output = outputs[:, -1, :]
final_output = get_linear_layer(last_rnn_output)

softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=y)
cross_entropy = tf.reduce_mean(softmax)
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32))) * 100

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

test_data = mnist.test.images[:batch_size].reshape(
    (-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

hidden_state: Tensor("strided_slice:0", shape=(?, 128), dtype=float32)


/Users/OG/Desktop/Study/jupyter/env/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [98]:
for i in range(3001):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, element_size))
    sess.run(train_step, feed_dict={_inputs: batch_x, y: batch_y})
    
    if i % 1000 == 0:
        acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
        loss = sess.run(cross_entropy, feed_dict={_inputs: batch_x, y: batch_y})
        print("Iter " + str(i) + ", Minibatch Loss= " +
              "{:.6f}".format(loss) + ", Training Accuracy= " +
              "{:.5f}".format(acc))

print("Testing Accuracy:", sess.run(
    accuracy, feed_dict={_inputs: test_data, y: test_label}))

Iter 0, Minibatch Loss= 2.303113, Training Accuracy= 9.37500
Iter 1000, Minibatch Loss= 0.185175, Training Accuracy= 92.18750
Iter 2000, Minibatch Loss= 0.044667, Training Accuracy= 100.00000
Iter 3000, Minibatch Loss= 0.037303, Training Accuracy= 100.00000
Testing Accuracy: 96.875
